In [1]:
import hopsworks
from dotenv import load_dotenv
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from hsml.schema import Schema
from hsml.model_schema import ModelSchema


load_dotenv()

False

In [2]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/693399
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
import hsfs

# Connection setup
# Connect to Hopsworks
api_key = os.getenv('hopsworks_api')
connection = hsfs.connection()
fs = connection.get_feature_store()

# Get feature view



Connected. Call `.close()` to terminate connection gracefully.


In [4]:
feature_view = fs.get_feature_view(
    name='tesla_stocks_fv',
    version=1
)

In [5]:
train_start = "2022-06-22"
train_end = "2023-12-31"

test_start = '2024-01-01'
test_end = "2024-05-03"


In [6]:
feature_view.create_train_test_split(
    train_start=train_start,
    train_end=train_end,
    test_start=test_start,
    test_end=test_end,
    data_format='csv',
    coalesce= True,
    statistics_config={'histogram':True,'correlations':True})

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/693399/jobs/named/tesla_stocks_fv_1_create_fv_td_06052024212158/executions
2024-05-06 23:23:21,130 WARNING: VersionWarning: Incremented version to `5`.



(5, <hsfs.core.job.Job at 0x194e21067d0>)

In [7]:
X_train, X_test, y_train, y_test = feature_view.get_train_test_split(5)

In [8]:
X_train

,date,ticker,sentiment
0,2022-12-14T00:00:00.000Z,TSLA,0.102207
1,2023-02-21T00:00:00.000Z,TSLA,0.155833
2,2023-08-17T00:00:00.000Z,TSLA,0.024046
3,2022-09-16T00:00:00.000Z,TSLA,0.087306
4,2023-08-28T00:00:00.000Z,TSLA,0.024046
...,...,...,...
378,2023-02-10T00:00:00.000Z,TSLA,0.155833
379,2023-05-08T00:00:00.000Z,TSLA,0.141296
380,2022-09-08T00:00:00.000Z,TSLA,0.087306
381,2023-07-06T00:00:00.000Z,TSLA,0.119444


In [9]:
X_test.head()

,date,ticker,sentiment
0,2024-04-16T00:00:00.000Z,TSLA,0.018769
1,2024-02-22T00:00:00.000Z,TSLA,0.212963
2,2024-02-13T00:00:00.000Z,TSLA,0.099363
3,2024-01-17T00:00:00.000Z,TSLA,0.099363
4,2024-02-16T00:00:00.000Z,TSLA,0.099363


In [10]:
X_test.tail()

,date,ticker,sentiment
80,2024-05-02T00:00:00.000Z,TSLA,0.001443
81,2024-04-02T00:00:00.000Z,TSLA,0.080911
82,2024-03-22T00:00:00.000Z,TSLA,0.080911
83,2024-01-02T00:00:00.000Z,TSLA,-0.122579
84,2024-02-26T00:00:00.000Z,TSLA,0.152764


In [11]:
X_train['date'] = pd.to_datetime(X_train['date']).dt.date
X_test['date'] = pd.to_datetime(X_test['date']).dt.date
X_train['date'] = pd.to_datetime(X_train['date'])
X_test['date'] = pd.to_datetime(X_test['date'])

In [12]:
X_train.head()

,date,ticker,sentiment
0,2022-12-14,TSLA,0.102207
1,2023-02-21,TSLA,0.155833
2,2023-08-17,TSLA,0.024046
3,2022-09-16,TSLA,0.087306
4,2023-08-28,TSLA,0.024046


In [13]:
# Extract the 'ticker' column
tickers = X_train[['ticker']]

# Initialize OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the 'ticker' column
ticker_encoded = encoder.fit_transform(tickers)

# Convert the encoded column into a DataFrame
ticker_encoded_df = pd.DataFrame(ticker_encoded.toarray(), columns=encoder.get_feature_names_out(['ticker']))

# Concatenate the encoded DataFrame with the original DataFrame
X_train = pd.concat([X_train, ticker_encoded_df], axis=1)

# Drop the original 'ticker' column
X_train.drop('ticker', axis=1, inplace=True)

In [14]:
X_train.head()

,date,sentiment,ticker_TSLA
0,2022-12-14,0.102207,1.0
1,2023-02-21,0.155833,1.0
2,2023-08-17,0.024046,1.0
3,2022-09-16,0.087306,1.0
4,2023-08-28,0.024046,1.0


In [15]:
tickers = X_test[['ticker']]

# Initialize OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the 'ticker' column
ticker_encoded_test = encoder.fit_transform(tickers)

# Convert the encoded column into a DataFrame
ticker_encoded_df_test = pd.DataFrame(ticker_encoded_test.toarray(), columns=encoder.get_feature_names_out(['ticker']))

# Concatenate the encoded DataFrame with the original DataFrame
X_test = pd.concat([X_test, ticker_encoded_df_test], axis=1)

# Drop the original 'ticker' column
X_test.drop('ticker', axis=1, inplace=True)

In [16]:
scaler = MinMaxScaler()

# Fit and transform the 'open' column
y_train['open_scaled'] = scaler.fit_transform(y_train[['open']])
y_train.drop('open', axis=1, inplace=True)

In [17]:
y_test['open_scaled'] = scaler.fit_transform(y_test[['open']])
y_test.drop('open', axis=1, inplace=True)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler  # Import StandardScaler from scikit-learn

def create_model(input_shape,
                 LSTM_filters=64,
                 dropout=0.1,
                 recurrent_dropout=0.1,
                 dense_dropout=0.5,
                 activation='relu',
                 depth=1):

    model = Sequential()

    # Input layer
    model.add(Input(shape=input_shape))

    if depth > 1:
        for i in range(1, depth):
            # Recurrent layer
            model.add(LSTM(LSTM_filters, return_sequences=True, dropout=dropout, recurrent_dropout=recurrent_dropout))

    # Recurrent layer
    model.add(LSTM(LSTM_filters, return_sequences=False, dropout=dropout, recurrent_dropout=recurrent_dropout))

    # Fully connected layer
    if activation == 'relu':
        model.add(Dense(LSTM_filters, activation='relu'))
    elif activation == 'leaky_relu':
        model.add(Dense(LSTM_filters))
        model.add(tf.keras.layers.LeakyReLU(alpha=0.1))

    # Dropout for regularization
    model.add(Dropout(dense_dropout))

    # Output layer for predicting one day forward
    model.add(Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='adam', loss='mse')

    return model

In [19]:
import numpy as np

# Assuming X_train['date'] column exists and is in datetime format
X_train['year'] = X_train['date'].dt.year
X_train['month'] = X_train['date'].dt.month
X_train['day'] = X_train['date'].dt.day

# Drop the original date column
X_train.drop(columns=['date'], inplace=True)

# Convert dataframe to numpy array
X_train_array = X_train.to_numpy()

# Reshape the array to have a shape suitable for LSTM
# Assuming each row represents a sample and each column represents a feature
# Reshape to [samples, timesteps, features]
X_train_array = np.expand_dims(X_train_array, axis=1)


2024-05-06 23:23:33,215 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



In [20]:
import numpy as np

# Convert DataFrame to numpy array
X_train_array = X_train.values

# Reshape X_train_array to add a time step dimension
X_train_reshaped = X_train_array.reshape(X_train_array.shape[0], 1, X_train_array.shape[1])

# Assuming X_train_reshaped shape is now (374, 1, 5)
input_shape = X_train_reshaped.shape[1:]

# Create the model
model = create_model(input_shape=input_shape)

In [21]:
model.fit(X_train_reshaped, y_train)

12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.5165


In [22]:
# Assuming X_test['date'] column exists and is in datetime format
X_test['year'] = X_test['date'].dt.year
X_test['month'] = X_test['date'].dt.month
X_test['day'] = X_test['date'].dt.day

# Drop the original date column
X_test.drop(columns=['date'], inplace=True)

# Convert dataframe to numpy array
X_test_array = X_test.to_numpy()

# Reshape the array to have a shape suitable for LSTM
# Assuming each row represents a sample and each column represents a feature
# Reshape to [samples, timesteps, features]
X_test_array = np.expand_dims(X_test_array, axis=1)


2024-05-06 23:23:37,549 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



In [30]:
y_pred = model.predict(X_test_array)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 237ms/step


In [31]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [37]:
import joblib
joblib.dump(model, 'LSTM_model.keras')

['LSTM_model.keras']

In [32]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Compute RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse_metrics = {"RMSE": rmse}
rmse_metrics


{'RMSE': 0.40675989895763576}

In [33]:
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

In [38]:
stock_pred_model = mr.tensorflow.create_model(
        name="stock_pred_model",
        metrics= rmse_metrics,
        model_schema=model_schema,
        description="Stock Market TSLA Predictor from News Sentiment",
    )

stock_pred_model.save('LSTM_model.keras')

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/291253 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/561 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/693399/models/stock_pred_model/4


Model(name: 'stock_pred_model', version: 4)